# --- Part Two ---

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

**What is the sum of all of the gear ratios in your engine schematic?**

**Notes**
* This should build nicely on part 1. Check for `*` instead of any symbol.
* The tricky part is determining if a `*` is a gear -- next to two parts, and not just one.
* **OR** do this in reverse...find the potential gears and then count how many parts it is next to.
* or really a combination of the two: identify the potential parts and the potential gears, and then figure out which gears touch exactly two parts

In [1]:
from utilities import get_lines

In [2]:
lines = get_lines('input')
len(lines)

140

In [3]:
max_index = len(lines[0])-1
max_index

139

In [4]:
max_line_num = len(lines)-1
max_line_num

139

In [5]:
import re

Need the index values of the part numbers in each line.

In [6]:
from collections import namedtuple

In [7]:
Match = namedtuple('Match','line_num span value')

In [8]:
parts = list()
#  will be a list of tuples (line index, number span, number)
pattern = re.compile('\d+')
for i, line in enumerate(lines):
    for match in re.finditer(pattern, line):
        parts.append(Match(i, match.span(), int(match.group())))
len(parts)

1189

In [9]:
gears = list()
#  will be a list of tuples for gears (line index, number span, '*')
pattern = re.compile('[*]')
for i, line in enumerate(lines):
    for match in re.finditer(pattern, line):
        gears.append(Match(i, match.span(), match.group()))
len(gears)

365

For each gear, check the parts in the lines above, below, and its own line. How many of their spans intersect with the span of the gear? 

In [10]:
def get_check_line_nums(line_num):
    '''helper function for range of lines to check'''
    if line_num==0:
        return (0,1)
    elif line_num==max_line_num:
        return(max_line_num-1, max_line_num)
    return (line_num-1, line_num+1)

In [11]:
def spans_intersect(span1, span2):
    '''helper function to see if two spans intersect'''
    index1 = set(range(span1[0],span1[1]+1))
    index2 = set(range(span2[0],span2[1]+1))
    intersection = len(index1.intersection(index2))
    if intersection>0:
        return True
    return False

In [12]:
def get_line_num_parts(line_num):
    '''helper function to get the parts from a line number'''
    return [part for part in parts if part.line_num==line_num]

In [13]:
def get_possible_parts(gear):
    '''returns a list of parts that are in the gear's range of line numbers'''
    check_lines = get_check_line_nums(gear.line_num)
    possible = list()
    for i in range(check_lines[0],check_lines[1]+1):
        num_parts = get_line_num_parts(i)
        if num_parts is not None:
            possible.extend(get_line_num_parts(i))
    return possible

In [14]:
def get_matching_parts(gear):
    '''returns a list of parts that match the gear'''
    possible = get_possible_parts(gear)
    matching_parts=list()
    for part in possible:
        if spans_intersect(gear.span, 
                           part.span):
            matching_parts.append(part)
    return matching_parts

In [15]:
def get_gear_ratio(gear):
    '''returns the gear ratio (defined in problem statement)'''
    gear_parts = get_matching_parts(gear)
    if len(gear_parts)==2:
        return gear_parts[0].value*gear_parts[1].value
    return 0

### After all of the above prep...
It is very quick to get the gear ratios for all the gears, and then add them up.

In [16]:
gear_ratios = list()
for gear in gears:
    gear_ratios.append(get_gear_ratio(gear))
sum(gear_ratios)

75519888

And with that, I earned a star! 🌟